In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [2]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Dikson,RU,2021-03-05 00:24:43,73.5069,80.5464,-29.51,82,3,13.04
1,1,Kirakira,SB,2021-03-05 00:24:44,-10.4544,161.9205,80.91,77,100,12.19
2,2,Port Hardy,CA,2021-03-05 00:24:46,50.6996,-127.4199,46.40,76,75,6.91
3,3,Jamestown,US,2021-03-05 00:20:58,42.0970,-79.2353,24.01,91,20,11.30
4,4,Grindavik,IS,2021-03-05 00:24:47,63.8424,-22.4338,39.20,93,90,4.61


In [3]:
city_data_df.dtypes

City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [4]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [11]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp], dissipating=False, max_intensity=300, point_radius=4)
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [12]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [13]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
1,1,Kirakira,SB,2021-03-05 00:24:44,-10.4544,161.9205,80.91,77,100,12.19
5,5,Saint-Pierre,RE,2021-03-05 00:24:48,-21.3393,55.4781,77.00,78,40,5.75
6,6,Sibu,MY,2021-03-05 00:24:49,2.3000,111.8167,75.20,94,20,4.61
7,7,Vila Velha,BR,2021-03-05 00:22:14,-20.3297,-40.2925,78.01,78,0,4.61
10,10,Rikitea,PF,2021-03-05 00:20:50,-23.1203,-134.9692,81.81,74,66,6.49
12,12,Vanimo,PG,2021-03-05 00:24:52,-2.6741,141.3028,83.86,68,100,5.39
16,16,Carnarvon,AU,2021-03-05 00:24:58,-24.8667,113.6333,75.20,94,12,9.22
23,23,Boa Vista,BR,2021-03-05 00:25:07,2.8197,-60.6733,86.00,58,75,6.91
43,43,Am Timan,TD,2021-03-05 00:24:09,11.0297,20.2827,77.20,18,76,8.63
44,44,Souillac,MU,2021-03-05 00:22:15,-20.5167,57.5167,75.20,83,40,5.75


In [14]:
preferred_cities_df.count()

City_ID       166
City          166
Country       165
Date          166
Lat           166
Lng           166
Max Temp      166
Humidity      166
Cloudiness    166
Wind Speed    166
dtype: int64

In [15]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
1,Kirakira,SB,80.91,-10.4544,161.9205,
5,Saint-Pierre,RE,77.00,-21.3393,55.4781,
6,Sibu,MY,75.20,2.3000,111.8167,
7,Vila Velha,BR,78.01,-20.3297,-40.2925,
10,Rikitea,PF,81.81,-23.1203,-134.9692,
12,Vanimo,PG,83.86,-2.6741,141.3028,
16,Carnarvon,AU,75.20,-24.8667,113.6333,
23,Boa Vista,BR,86.00,2.8197,-60.6733,
43,Am Timan,TD,77.20,11.0297,20.2827,
44,Souillac,MU,75.20,-20.5167,57.5167,


In [16]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key}

In [19]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [22]:
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
1,Kirakira,SB,80.91,-10.4544,161.9205,SanBiz Lodge
5,Saint-Pierre,RE,77.00,-21.3393,55.4781,Lindsey Hôtel
6,Sibu,MY,75.20,2.3000,111.8167,RH HOTEL (4 Star)
7,Vila Velha,BR,78.01,-20.3297,-40.2925,Quality Suites Vila Velha
10,Rikitea,PF,81.81,-23.1203,-134.9692,Pension Maro'i
12,Vanimo,PG,83.86,-2.6741,141.3028,Vahmoneh Lodge Vanimo
16,Carnarvon,AU,75.20,-24.8667,113.6333,Hospitality Carnarvon
23,Boa Vista,BR,86.00,2.8197,-60.6733,Aipana Plaza Hotel
43,Am Timan,TD,77.20,11.0297,20.2827,Palais du chef Canton Salamt
44,Souillac,MU,75.20,-20.5167,57.5167,Shanti Maurice Resort & Spa


In [24]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [25]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

In [26]:
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [27]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))